## mecabで分かち書きしながら、tokenizerを作成する

In [ ]:
# google driveのマウント
# google driveの方のみ実行　
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
BASE_PATH = "/home/ubuntu/Tanuki_pretraining/learning_tokenizer" #自身の作業用フォルダーを指定
%cd {BASE_PATH}

/home/ubuntu/Tanuki_pretraining/learning_tokenizer


In [3]:
!sudo apt install -y mecab libmecab-dev mecab-ipadic-utf8

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libmecab-dev is already the newest version (0.996-14build9).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-3).
mecab is already the newest version (0.996-14build9).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


jsonlファイルをtxtファイルに変換

In [4]:
import json
target_path="./learning_corpus/corpus_scale_200.jsonl" #自身で用意したコーパス
num = 0
max_len = 0
with open("./preprocess_data/corpus_text.txt","w") as o:
    o.write("")
with open("./preprocess_data/corpus_text.txt","a") as o:
    with open(target_path, "r") as f:
        for line in f:
            num += 1
            jsonl_line = (json.loads(line)["text"])
            o.write(jsonl_line + "\n")
            if len(jsonl_line) > max_len:
                max_len = len(jsonl_line)
                max_line = jsonl_line

print(f'num: {num}, max_len: {max_len}' )

num: 1216549, max_len: 877744


In [ ]:
print (max_line)

In [6]:
#わかちがき
!mecab -F"%M||||" -E"\n" -b 400000 < ./preprocess_data/corpus_text.txt  > ./preprocess_data/corpus_text.tok
#!mecab -F"%M||||" -E"\n" -b 100000 < ./preprocess_data/corpus_text.txt  > ./preprocess_data/corpus_text_100K.tok

input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.


In [7]:
!head -n100 ./preprocess_data/corpus_text.tok

ただし||||、||||本来||||で||||あれ||||ば||||特別||||法人||||課税||||と||||いっ||||て||||、||||個人||||型||||も||||企業||||型||||も||||積立||||金||||の||||1||||.||||173||||%||||が||||課税||||さ||||れる||||こと||||に||||なっ||||て||||いる||||の||||です||||。||||これ||||は||||現在||||凍結||||さ||||れ||||て||||いる||||措置||||に||||なり||||ます||||。||||
祖母||||と||||霊||||能||||者||||の||||おかげ||||だ||||と||||感謝||||し||||て||||い||||ます||||よ||||ね||||。||||この||||ブレスレット||||は||||とても||||可愛く||||て||||一目||||惚れ||||し||||た||||の||||です||||が||||、||||それ||||を||||右腕||||に||||付ける||||と||||急||||に||||右腕||||が||||重たく||||なっ||||て||||気分||||が||||沈む||||よう||||に||||なり||||まし||||た||||。||||
Bayes|||| and|||| his|||| ||||Theorem||||

My|||| earlier|||| post|||| on|||| Bayesian|||| probability|||| seems|||| to|||| have|||| generated|||| quite|||| a|||| lot|||| of|||| readers||||,|||| so|||| this|||| lunchtime|||| I|||| thought|||| I||||’||||d|||| add|||| a|||| little|||| bit|||| of|||| background||||.|||| The|||| previous|||| discussion|||| started|||| from|||| the|||| result||||

$||||P||||(||||B|||||||||

In [8]:
from types import SimpleNamespace
import sentencepiece as spm
import sys
import yaml
import os
sys.path.append(BASE_PATH)
from config.special_token_list import *

def yaml_to_namespace(yaml_path):
    with open(yaml_path, 'r') as file:
        # YAMLファイルを辞書として読み込む
        data = yaml.safe_load(file)
        # 辞書をSimpleNamespaceに変換
        return recursive_namespace(data)

def recursive_namespace(data):
    if isinstance(data, dict):
        # 再帰的に辞書の各要素をSimpleNamespaceに変換
        return SimpleNamespace(**{k: recursive_namespace(v) for k, v in data.items()})
    elif isinstance(data, list):
        # リストの要素も変換
        return [recursive_namespace(v) for v in data]
    else:
        # その他のデータ型はそのまま返す
        return data

args = yaml_to_namespace('./config/config.yaml')

#args.input="./preprocess_data/corpus_text.tok"
#args.vocab_size=3000

In [9]:
spm.SentencePieceTrainer.train(
        input=args.input,
        model_prefix=args.model_prefix,
        vocab_size=args.vocab_size,
        character_coverage=args.character_coverage,
        model_type=args.model_type,
        num_threads=args.num_threads,
        train_extremely_large_corpus=args.train_extremely_large_corpus,
        user_defined_symbols=[
            BOS_TOKEN,
            EOS_TOKEN,
            PAD_TOKEN,
            CLS_TOKEN,
            SEP_TOKEN,
            EOD_TOKEN,
            MASK_TOKEN,
            NEWLINE_TOKEN,
            EXTRA_TOKEN1,
            EXTRA_TOKEN2,
            EXTRA_TOKEN3,
            EXTRA_TOKEN4,
        ],  # Note: `NEWLINE_TOKEN` is needed in `user_defined_symbols`.
        byte_fallback=True,
        split_digits=True,
        allow_whitespace_only_pieces=True,
        remove_extra_whitespaces=False,
        pretokenization_delimiter="||||",
        # google_colab用
        input_sentence_size=1000000,
    )

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./preprocess_data/corpus_text.tok
  input_format: 
  model_prefix: tokenizer
  model_type: UNIGRAM
  vocab_size: 65000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 1000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 8
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: ||||
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  user_defined_symbols: <s>
  user_defined_symbols: </s>
  user_defined_symbols: <pad>
  user_defined_symbols: <CLS>
  user_defined_symbols: <SEP>
  user_defined_symbols: <EOD>
  user_defined_symbols: <MASK>
  user_defined_symbols: 

  user_defined_symbols: <llm-code>
  user_defined_symbols: </llm-code>
  user_defined_symbol

In [10]:
import sentencepiece as spm
#test
model_path="./tokenizer.model"
sp = spm.SentencePieceProcessor(model_file=model_path)

In [11]:

text=" 価格詳細, 日本の正式代理店で購入する必要がある"
text="he is a good man"
text="import pandas as pd  from tqdm import tqdm"
text="""
def yaml_to_namespace(yaml_path):
    with open(yaml_path, 'r') as file:
        # YAMLファイルを辞書として読み込む
        data = yaml.safe_load(file)
        # 辞書をSimpleNamespaceに変換
        return recursive_namespace(data)
"""

text="これはテストです。"
text="吾輩は猫である｡名前はまだない"
text="吾輩は猫である｡\n名前はまだない"
(sp.encode(text, out_type=str)),(sp.encode(text))

(['▁', '吾', '輩', 'は', '猫', 'である', '。', '\n', '名前', 'は', 'まだ', 'ない'],
 [272, 5906, 1903, 276, 1820, 390, 271, 8, 1265, 276, 759, 320])

In [12]:
t="ああ\nああ"
tokens=sp.encode(t)
t,tokens,sp.decode(tokens)

('ああ\nああ', [7640, 8, 2750], 'ああ\nああ')